In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install efficientnet

In [ ]:
import pandas as pd
from tensorflow.keras.optimizers import RMSprop,Adam
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras import Model
from os import getcwd
from keras.layers import Dense, Activation, BatchNormalization
import tensorflow as tf
import pandas as pd 
import numpy as np 
from sklearn.model_selection import StratifiedKFold
import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa 
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import keras 
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import efficientnet.tfkeras as efn

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
earlyStopping = EarlyStopping(monitor='val_accuracy', patience = 15, verbose=0, mode='max')
mcp_save_xc = ModelCheckpoint(filepath='mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='max')
mcp_save_ens = ModelCheckpoint(filepath='mdl_wts_en.h5', save_best_only=True, monitor='val_accuracy', mode='max')
mcp_save_ens1 = ModelCheckpoint(filepath='mdl_wts_en1.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, min_delta=1e-4, mode='min')

In [ ]:
train_df = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")

In [ ]:
train_df['label']=train_df['label'].astype(str)

In [ ]:
SEED=42
TRAIN_DIR='../input/cassava-leaf-disease-classification/train_images'
# TEST_DIR='data/DATA/TEST'
IMG_SIZE=(300,300)
BATCHSIZE=16
INPUT_SHAPE=(300,300,3)
EPOCHS=70
LR=1e-4

In [ ]:
train_datagen=ImageDataGenerator(rescale=1/255,
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    validation_split=0.05,
    #zoom_range=.2,

    horizontal_flip=True)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
    directory = TRAIN_DIR,
    x_col="image_id",
    y_col="label",
    batch_size = BATCHSIZE,
    subset = 'training',
    class_mode='categorical',
    shuffle = True,
    target_size=(300,300)
)


validation_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
    directory = TRAIN_DIR,
    x_col="image_id",
    y_col="label",
    shuffle = True,
    batch_size = BATCHSIZE,
    subset = 'validation',
    class_mode='categorical',
    target_size=(300,300)
)

In [ ]:
eff=efn.EfficientNetB3(weights='imagenet',include_top=False,input_shape=INPUT_SHAPE)

In [ ]:
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
model=Sequential()
model.add(eff)
model.add(Dropout(0.5))
model.add(GlobalAveragePooling2D())
model.add(Dense(5,activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(lr=LR),loss='categorical_crossentropy',metrics=[f1_metric])

In [ ]:
redlr=ReduceLROnPlateau(monitor='val_loss',patience=3,verbose=1)
chkpt=ModelCheckpoint('best.h5',verbose=1,monitor='val_loss',save_best_only=True)
es=EarlyStopping(patience=8,verbose=1,restore_best_weights=True)

In [ ]:
history=model.fit(train_generator,validation_data=validation_generator,epochs=15,callbacks=[redlr,chkpt,es],verbose=1)

In [ ]:
from PIL import Image

In [ ]:
TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'
test_images = os.listdir(TEST_DIR)
predictions = []

for image in test_images:
    img = Image.open(TEST_DIR + image)
    img = img.resize(IMG_SIZE)
    img = np.expand_dims(img, axis=0)
    predictions.extend(model.predict(img).argmax(axis = 1))

In [ ]:
predictions

In [ ]:
sub = pd.DataFrame({'image_id': test_images, 'label': predictions})
display(sub)
sub.to_csv('submission.csv', index = False)